# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal
import shutil

#### Creating list of filepaths to process original event csv data files

In [2]:

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list=[]

#DELETE FOLDER IF EXIST TO SELECT ONLY NEEDED DATA
try:
    shutil.rmtree(filepath+'/.ipynb_checkpoints')
except Exception as e :
    print(e)
    
#Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):   
# join the file path and roots with the subdirectories using glob
    file_path_list =(glob.glob(os.path.join(root,'*events.csv')))
    #print(file_path_list)
    #print(len(file_path_list))

[Errno 2] No such file or directory: '/home/workspace/event_data/.ipynb_checkpoints'


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:    
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) 
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            



# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
#connecting to local machine casendra
from cassandra.cluster import Cluster
cluster = Cluster()

#  Creating a session
session = cluster.connect()

#### Create Keyspace

In [6]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS SPARKIFY
    WITH REPLICATION=
    {'class':'SimpleStrategy','replication_factor':1}
    """)

except Exception as e:
    print(e)
    

#### Set Keyspace

In [7]:

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#so our data will by partitioned by sessionId and clustered by itemInSession
try:
    session.execute('DROP TABLE IF EXISTS song_hist') 
except Exception as e :
    print(e)
try:
    query='CREATE TABLE song_hist'
    query+="""(sess_id int,
    user_id int,
    sess_item int ,
    first_name varchar,
    last_name varchar ,
    gender varchar ,
    location text,
    level varchar,
    artist varchar,
    song varchar,
    length double,
    primary key (sess_id,sess_item)
    )"""
    session.execute(query) 
except Exception as e :
    print(e)                  

In [10]:
#test record for csv
#with open('event_datafile_new.csv', encoding = 'utf8') as f:
#    csvreader = csv.reader(f)
#    #print header
#    print(next(csvreader))
#    #print first line
#    print(next(csvreader))
#    f.close()
    

In [9]:
#reading the smaller file and inserting data in table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """INSERT INTO song_hist (sess_id,
        user_id  ,
        sess_item ,
        first_name ,
        last_name ,
        gender  ,
        location,
        level,
        artist,
        song ,
        length)"""
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[1], line[4], line[2], line[7], line[6], line[0], line[9], Decimal(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:

#verify number of records in table 

#try:
#    query='SELECT COUNT (*) FROM song_hist'
#    rows=session.execute(query) 
#except Exception as e :
#    print(e)   
#for row in rows:
#    print (row.count)
    
#Query1: select the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4 
    
try:
    query='SELECT artist,song ,length FROM song_hist where sess_id=338 and sess_item=4'
    rows=session.execute(query) 
except Exception as e :
    print(e)
    
for row in rows:
    print (row.artist,row.song,row.length)
    


Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#So our table will be partitioned by userId and clustered by SessionId and for uniqueness we will add  items in Session to  Clustering columns

try:
    session.execute('DROP TABLE IF EXISTS song_of_user') 
except Exception as e :
    print(e)
try:
    query='CREATE TABLE song_of_user'
    query+="""(sess_id int,
    user_id int,
    sess_item int ,
    first_name varchar,
    last_name varchar ,
    gender varchar ,
    location text,
    level varchar,
    artist varchar,
    song varchar,
    length double,
    primary key (user_id,sess_id,sess_item)
    )"""
    session.execute(query) 
except Exception as e :
    print(e)  
                    

In [13]:
#reading the smaller file and inserting data in table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """INSERT INTO song_of_user (sess_id,
        user_id  ,
        sess_item ,
        first_name ,
        last_name ,
        gender  ,
        location,
        level,
        artist,
        song ,
        length)"""
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[1], line[4], line[2], line[7], line[6], line[0], line[9], Decimal(line[5])))

In [14]:
#verify number of records in table 

#try:
#    query='SELECT COUNT (*) FROM song_of_user'
#    rows=session.execute(query) 
#except Exception as e :
#    print(e)   
#for row in rows:
#    print (row.count)
    
#Query2:Select only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
try:
    query='SELECT artist,sess_item,song,first_name,last_name FROM song_of_user where user_id=10 and sess_id=182 '
    rows=session.execute(query) 
except Exception as e :
    print(e)
    
for row in rows:
    print (row.artist,row.sess_item,row.song,row.first_name,row.last_name)

Down To The Bone 0 Keep On Keepin' On Sylvie Cruz
Three Drives 1 Greece 2000 Sylvie Cruz
Sebastien Tellier 2 Kilometer Sylvie Cruz
Lonnie Gordon 3 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#So our table will be partitioned by song and for uniqueness we will add sessionId and item in session to clustering columns to identify row
try:
    session.execute('DROP TABLE IF EXISTS user_of_song ') 
except Exception as e :
    print(e)
try:
    query='CREATE TABLE user_of_song '
    query+="""(sess_id int,
    user_id int,
    sess_item int ,
    first_name varchar,
    last_name varchar ,
    gender varchar ,
    location text,
    level varchar,
    artist varchar,
    song varchar,
    length double,
    primary key (song,sess_id,sess_item)
    )"""
    session.execute(query) 
except Exception as e :
    print(e)  

                    

In [16]:
#reading the smaller file and inserting data in table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """INSERT INTO user_of_song  (sess_id,
        user_id  ,
        sess_item ,
        first_name ,
        last_name ,
        gender  ,
        location,
        level,
        artist,
        song ,
        length)"""
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[1], line[4], line[2], line[7], line[6], line[0], line[9], Decimal(line[5])))

In [17]:
#verify number of records in table 

#try:
#    query='SELECT COUNT (*) FROM user_of_song '
#    rows=session.execute(query) 
#except Exception as e :
#    print(e)   
#for row in rows:
#    print (row.count)


#Query3:Select  every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
    
try:
    query="SELECT first_name,last_name FROM user_of_song  where song='All Hands Against His Own'"
    rows=session.execute(query) 
except Exception as e :
    print(e)
    
for row in rows:
    print (row.first_name,row.last_name)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables before closing out the sessions

In [18]:
##Dropping tables
try:
    session.execute('DROP TABLE IF EXISTS song_hist') 
except Exception as e :
    print(e)

try:
    session.execute('DROP TABLE IF EXISTS song_of_user') 
except Exception as e :
    print(e)
    
try:
    session.execute('DROP TABLE IF EXISTS user_of_song') 
except Exception as e :
    print(e)


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()